In [12]:
import requests
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from PIL import Image
from io import BytesIO
from transformers import AutoProcessor, AutoModelForUniversalSegmentation
import csv

def sky(location):
    API_KEY = 'YOUR_API_KEY'  # Replace with your API key
    fov = 120
    pitch = 0

    # Retrieve the panorama ID for the location
    metadata_url = rf'https://maps.googleapis.com/maps/api/streetview/metadata?location={location}&key={API_KEY}'
    metadata_response = requests.get(metadata_url).json()
    display(metadata_response)
    pano_id = metadata_response['pano_id']

    # Capture multiple images from different angles around the location
    image_list = []
    for heading in range(0, 360, fov):
        url = f'https://maps.googleapis.com/maps/api/streetview?size=600x1000&pano={pano_id}&fov={fov}&heading={heading}&pitch={pitch}&key={API_KEY}'
        response = requests.get(url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            image_list.append(image)
        else:
            print(f'Image request failed for heading {heading}.')

    # Create a blank canvas to stitch the images
    canvas_width = image_list[0].width * len(image_list)
    canvas_height = image_list[0].height
    canvas = Image.new('RGB', (canvas_width, canvas_height))

    # Stitch the images together horizontally
    for i, image in enumerate(image_list):
        canvas.paste(image, (i * image.width, 0))

    # Save the canvas image with a unique name based on the location
    canvas.save(f'G:\Mi unidad\AI in urb\images segmented\canvas_image_{location.replace(",", "_")}.jpg')

    images_for_segmentation = [canvas]

    # Initialize model modules
    model = AutoModelForUniversalSegmentation.from_pretrained("shi-labs/oneformer_coco_swin_large")
    processor = AutoProcessor.from_pretrained("shi-labs/oneformer_coco_swin_large")

    def tensor_to_segmented_image(segmented_tensor, num_classes=None):
        # Converting PyTorch tensor to a NumPy array
        segmented_array = segmented_tensor.numpy().astype(np.uint8)

        # Assigning a color to each class
        unique_values = np.unique(segmented_array)
        if num_classes is None:
            num_classes = len(unique_values)
        colors = np.random.randint(0, 255, size=(num_classes, 3), dtype=np.uint8)

        # Map unique values in the segmented_array to indices
        value_to_index_map = {value: index for index, value in enumerate(unique_values)}
        mapped_array = np.vectorize(value_to_index_map.get)(segmented_array)

        # Apply colors to the mapped_array
        colored_segmentation = colors[mapped_array]

        # Converting NumPy array to PIL Image
        segmented_image = Image.fromarray(colored_segmentation)
        return segmented_image

    def draw_panoptic_segmentation(segmentation, segments_info):
        # Create a new image with the same shape as the segmentation
        image = np.zeros(segmentation.shape + (3,), dtype=np.uint8)

        # Find the segment corresponding to the 'sky-other-merged' label
        sky_segment_id = None
        for segment in segments_info:
            segment_id = segment['id']
            segment_label_id = segment['label_id']
            segment_label = model.config.id2label[segment_label_id]

            if segment_label == 'sky-other-merged':
                sky_segment_id = segment_id
                break

        # Paint the sky segment blue and the rest black
        if sky_segment_id is not None:
            image[segmentation == sky_segment_id] = [0, 0, 255]  # Paint sky segment blue

        # Calculate percentage of 'sky-other-merged' label
        total_pixels = image.size // 3  # Divide by 3 for RGB image
        label_pixels = np.count_nonzero(np.all(image == [0, 0, 255], axis=-1))
        label_percentage = (label_pixels / total_pixels) * 100

        return label_percentage, image

    def getImageData(img, model, processor):
        # Encode image
        image_inputs = processor(images=img, task_inputs=["panoptic"], return_tensors="pt")

        # Forward pass
        with torch.no_grad():
            outputs = model(**image_inputs)

        # Get results
        img_segmentation_results = processor.post_process_panoptic_segmentation(outputs, target_sizes=[img.size[::-1]])[0]

        # Call result processing function
        label_percentage, image_out = draw_panoptic_segmentation(**img_segmentation_results)

        return label_percentage, image_out

    segmented_images = []
    percentages = []
    for cur_image in images_for_segmentation:
        label_percentage, segmented_image = getImageData(cur_image, model, processor)
        segmented_images.append(segmented_image)
        percentages.append(label_percentage)

    # Save the segmented images with unique names based on the location
    for i, segmented_image in enumerate(segmented_images):
        segmented_image_pil = Image.fromarray(segmented_image)
        segmented_image_pil.save(f'G:\Mi unidad\AI in urb\images segmented\segmented_image_{location.replace(",", "_")}_{i}.jpg')

    # Return the percentages for each location
    return percentages




In [28]:
import pandas as pd
import csv
import geopandas as gpd

# Assuming you have a GeoDataFrame named 'gdf' with 'x' and 'y' fields
gdf = gpd.read_file(rf"G:\Mi unidad\AI in urb\NEWBATCH.geojson")
display(gdf)

# Extract coordinates without parentheses
coordinates = [(row['Y'], row['X']) for _, row in gdf.iterrows()]
coordinates = [','.join(map(str, coord)) for coord in coordinates]
display(coordinates)


,distance,angle,X,Y,geometry
0,0.0,314.746152,2.175794,41.374298,POINT (2.17579 41.37430)
1,20.0,314.746152,2.175623,41.374424,POINT (2.17562 41.37442)
2,40.0,314.746152,2.175451,41.374550,POINT (2.17545 41.37455)
3,60.0,289.663246,2.175230,41.374615,POINT (2.17523 41.37461)
4,80.0,274.946139,2.174999,41.374654,POINT (2.17500 41.37465)
...,...,...,...,...,...
877,2300.0,315.440130,2.196686,41.418188,POINT (2.19669 41.41819)
878,2320.0,315.440130,2.196517,41.418315,POINT (2.19652 41.41832)
879,2340.0,315.440130,2.196347,41.418442,POINT (2.19635 41.41844)
880,2360.0,315.440130,2.196178,41.418570,POINT (2.19618 41.41857)


['41.374298,2.175794',
 '41.374424,2.175623',
 '41.37455,2.175451',
 '41.374615,2.17523',
 '41.374654,2.174999',
 '41.374667,2.17476',
 '41.374681,2.174522',
 '41.374695,2.174284',
 '41.374709,2.174045',
 '41.374723,2.173807',
 '41.374736,2.173568',
 '41.37475,2.17333',
 '41.374764,2.173091',
 '41.374778,2.172853',
 '41.374792,2.172614',
 '41.374806,2.172376',
 '41.374819,2.172138',
 '41.374833,2.171899',
 '41.374847,2.171661',
 '41.374861,2.171422',
 '41.374875,2.171184',
 '41.374888,2.170945',
 '41.374902,2.170707',
 '41.374916,2.170468',
 '41.374927,2.17023',
 '41.374937,2.169991',
 '41.374947,2.169752',
 '41.374956,2.169513',
 '41.374966,2.169274',
 '41.374967,2.169035',
 '41.374967,2.168796',
 '41.374968,2.168557',
 '41.374968,2.168318',
 '41.374968,2.168079',
 '41.374969,2.16784',
 '41.374969,2.1676',
 '41.37497,2.167361',
 '41.37497,2.167122',
 '41.37497,2.166883',
 '41.374971,2.166644',
 '41.374971,2.166405',
 '41.374971,2.166165',
 '41.374972,2.165926',
 '41.374972,2.165687',


In [29]:
# Iterate over each coordinate and call the sky() function
all_percentages = []
for i, coordinate in enumerate(coordinates):
    display(coordinate)
    percentages = sky(coordinate) 
    all_percentages.append(percentages)


'41.374298,2.175794'

{'copyright': '© Google',
 'date': '2023-03',
 'location': {'lat': 41.37432716683941, 'lng': 2.175842180866574},
 'pano_id': 'jx-TJm3l7cu_F6tmz3hN9Q',
 'status': 'OK'}

C:\Users\Maria Augusta\anaconda3\lib\site-packages\transformers\models\oneformer\image_processing_oneformer.py:417: FutureWarning: The `reduce_labels` argument is deprecated and will be removed in v4.27. Please use `do_reduce_labels` instead.
  warnings.warn(
`label_ids_to_fuse` unset. No instance will be fused.


'41.374424,2.175623'

{'copyright': '© Google',
 'date': '2023-04',
 'location': {'lat': 41.37449133913471, 'lng': 2.175671590452018},
 'pano_id': 'wviNqBnCPedUQEd26uOFrg',
 'status': 'OK'}

...

{'copyright': '© Google',
 'date': '2022-01',
 'location': {'lat': 41.41855249211658, 'lng': 2.196187488964427},
 'pano_id': '-gCPh1ACt2THGK07B0RIsg',
 'status': 'OK'}

`label_ids_to_fuse` unset. No instance will be fused.


'41.418697,2.196008'

{'copyright': '© Google',
 'date': '2022-01',
 'location': {'lat': 41.41869289056368, 'lng': 2.196000917587623},
 'pano_id': 'JsiWo1Pg_3p3FkhMYjcd9g',
 'status': 'OK'}

`label_ids_to_fuse` unset. No instance will be fused.


In [30]:
# Save all the percentages to a CSV file
with open(rf'G:\Mi unidad\AI in urb\images segmented\new batch\sky_percentages.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['X Coordinate', 'Y Coordinate', 'Segmented Image', 'Sky Percentage'])
    for i, percentages in enumerate(all_percentages):
        coordinate = coordinates[i].split(',')
        x_coordinate, y_coordinate = float(coordinate[1]), float(coordinate[0])
        for j, percentage in enumerate(percentages):
            writer.writerow([x_coordinate, y_coordinate, f'G:\Mi unidad\AI in urb\images segmented\segmented_image_{x_coordinate}_{y_coordinate}_{j}.jpg', percentage])
